In [ ]:
!pip install -qq transformers

Este código carga los conjuntos de datos de entrenamiento y prueba desde los archivos CSV utilizando Pandas. Luego codifica las etiquetas categóricas como números enteros utilizando un objeto LabelEncoder. Se define una clase personalizada para representar los conjuntos de datos como objetos Dataset de PyTorch. Esta clase tokeniza el texto utilizando el tokenizer BERT y devuelve los tensores de entrada para el modelo junto con la etiqueta correspondiente. Luego se crean objetos Dataset para los conjuntos de datos de entrenamiento y prueba y se crean DataLoaders para ellos. El resto del código es similar al ejemplo anterior.

Una vez termina el entrenamiento del modelo, se guarda un csv con los estadísticos importantes para su posterior comparación:
- datetime de la realizacion de la prueba
- tamaño de la muestra
- modelo bert utilizado.
- Tiempo de entrenamiento
- memoria utilizada
- Si ha utilizado GPU
- Uso de memoria de GPU
- estadísticos de precisión del accuracy_score.

In [ ]:
import urllib.request

# Definir el nombre del modelo BERT utilizado
bert_model_name = 'bert-base-uncased'

# Cargar el tokenizer y el modelo pre-entrenado de BERT
tokenizer = BertTokenizer.from_pretrained(bert_model_name)
model = BertForSequenceClassification.from_pretrained(bert_model_name, num_labels=4)



train_data = pd.read_csv ("./sample_data/news_category_train.csv",sep=",",engine="c",error_bad_lines=False)
print(train_data.head())

test_data = pd.read_csv ("./sample_data/news_category_test.csv",sep=",",engine="c",error_bad_lines=False)


# Instalar Spark NLP
!pip install spark-nlp==3.3.1

# Importar las librerías necesarias
from pyspark.ml import Pipeline
from sparknlp.annotator import *
from sparknlp.base import *
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
import time
import psutil
from datetime import datetime

# Definir el nombre del modelo BERT utilizado
bert_model_name = 'bert_base_cased'

# Cargar los conjuntos de datos de entrenamiento y prueba desde los archivos CSV
base_url = "https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Public/data"
train_url = base_url+"/news_category_train.csv"
test_url = base_url+"/news_category_test.csv"

urllib.request.urlretrieve(train_url, "./sample_data/news_category_train.csv")
urllib.request.urlretrieve(test_url, "./sample_data/news_category_test.csv")


train_data = spark.read.option("header", True).csv("./sample_data/news_category_train.csv")
train_data.show(truncate=False)
test_data = spark.read.option("header", True).csv("./sample_data/news_category_test.csv")

# Codificar las etiquetas categóricas como números enteros
label_indexer = StringIndexer(inputCol="category", outputCol="label").fit(train_data)
train_data = label_indexer.transform(train_data)
test_data = label_indexer.transform(test_data)

# Definir el pipeline de procesamiento de texto utilizando Spark NLP
document_assembler = DocumentAssembler().setInputCol("description").setOutputCol("document")
tokenizer = Tokenizer().setInputCols(["document"]).setOutputCol("token")
bert_embeddings = BertEmbeddings.pretrained(bert_model_name, "en").setInputCols(["document", "token"]).setOutputCol("embeddings")
embeddings_sentence = SentenceEmbeddings().setInputCols(["document", "embeddings"]).setOutputCol("sentence_embeddings").setPoolingStrategy("AVERAGE")
classifier_dl = ClassifierDLApproach().setInputCols(["sentence_embeddings"]).setOutputCol("prediction").setLabelColumn("label").setMaxEpochs(5).setEnableOutputLogs(True)
nlp_pipeline = Pipeline(stages=[document_assembler, tokenizer, bert_embeddings, embeddings_sentence, classifier_dl])

# Entrenar el modelo y guardar estadísticas en un archivo CSV
with open('stats.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(['experiment_name','datetime', 'sample_size', 'bert_model', 'training_time', 'memory_usage', 'gpu_usage', 'gpu_memory_usage', 'accuracy'])

    start_time = time.time()
    nlp_model = nlp_pipeline.fit(train_data)
    end_time = time.time()
    training_time = end_time - start_time

    memory_usage = psutil.virtual_memory().percent

    gpu_usage = torch.cuda.is_available()
    gpu_memory_usage = torch.cuda.memory_allocated() if gpu_usage else 0

    # Evaluar el modelo en el conjunto de datos de prueba
    predictions = nlp_model.transform(test_data)
    evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
    accuracy = evaluator.evaluate(predictions)

    writer.writerow(['Spark NLP - AG News',datetime.now(), train_data.count(), bert_model_name, training_time, memory_usage, gpu_usage, gpu_memory_usage, accuracy])





KeyboardInterrupt: ignored

Se crea una función `predict` para hacer una predicción con el modelo BERT que se ha entrenado utilizando un texto de entrada X. La función toma como argumentos el modelo y el texto de entrada

In [ ]:
def predict(model, text):
    # Crear un DataFrame de Spark con el texto de entrada
    data = spark.createDataFrame([(text,)], ['description'])

    # Aplicar el pipeline de procesamiento de texto al texto de entrada
    result = model.transform(data)

    # Obtener la etiqueta predicha
    predicted_label = result.select('prediction').collect()[0]['prediction']
    predicted_label = label_indexer.labels[int(predicted_label)]

    return predicted_label

 ## prediccion una palabra

In [ ]:
text = 'aftermatch everything went dark, 1 death 2 injuries'
predicted_label = predict(nlp_model, text)
print(f'Texto: {text}')
print(f'Etiqueta predicha: {predicted_label}')